# Open Soure Threat Intel Lookups & Enrichment

## Description :
This notebook is example implementaion of open source threat intel lookup using requests API.

## Import Python Packages

In [ ]:
# Imports
from IPython import get_ipython
from IPython.display import display, HTML

import pandas as pd

import requests, json
from requests.exceptions import HTTPError
from requests.auth import HTTPBasicAuth

# ThreatIntel Providers Research

## A few examples

 - Shodan :
     - API Registeration : https://account.shodan.io/register
 - Alienvault OTX
    - API Registeration : https://otx.alienvault.com/api
 - IBM Xforce
    - API Registration : https://www.ibm.com/account/reg/signup?formid=urx-30243
 - ThreatMiner
    - API Registration : https://threatminer.org/api.php
 - ThreatCrowd
    - API Registration : https://github.com/AlienVault-OTX/ApiV2
 - RiskIQ
    - API Registration : https://community.riskiq.com/registration
 - Pulsedive
    - API Registration : https://pulsedive.com/auth/new.php
 - Censys
    - API Registration : https://censys.io/register
 - SANS DShield
    - API Registration : https://isc.sas.edu/api

In [ ]:
OTX_API='<otx-key>'

# Lookup Intel From Providers

In [ ]:
def otx_iplookup(string,path="v1/indicators/IPv4/",section="/general"):
    """ Perform whois Lookups on IP Address using Alienvault OTX API """
    API_BASE_URL="https://otx.alienvault.com/api/"
    try:
        result =  requests.get(API_BASE_URL+path+string+section)
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err

def dshield_iplookup(string,outputformat="?json"):
    """ Perform whois Lookups on IP Address using SANS DShield API """
    API_BASE_URL="https://isc.sans.edu/api/ip/"
    try:
        result =  requests.get(API_BASE_URL+string+outputformat)
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err

def threatminer_iplookup(string,rt=1):
    """ Perform whois Lookups on IP Address using Threatminer API """
    API_BASE_URL="https://api.threatminer.org/v2/host.php"
    try:
        result =  requests.get(API_BASE_URL,params = {"q": string,"rt":rt})
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err

def pulsedive_iplookup(string):
    """ Perform whois Lookups on IP Address using Pulsedive API """
    API_BASE_URL="https://pulsedive.com/api/info.php"
    try:
        result =  requests.get(API_BASE_URL,params = {"indicator": string,"key":""})
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err

## PulseDive

In [ ]:
pulsediveresults = pulsedive_iplookup("192.64.119.63")
# Code To extract essential details from raw json response
if len(pulsediveresults['threats']) > 0 :
    print(f"Raw JSON - {pulsediveresults}\n\n")
    display(HTML('<div class="alert alert-danger"> Match Found via Pulsedive API </div>'))
    print(f"{len(pulsediveresults['threats'])} Threats Idenfied")
    print(f"\n PulseDive Tags or Categories Identified:")
    for item in pulsediveresults['threats']:print(item['category'])
    print(f"\n PulseDive Reported Related Threats Description:")
    for item in pulsediveresults['threats']:print(item['name'])
    print(f"\n PulseDive Feed Names:")
    for item in pulsediveresults['feeds']:print(item['name'])
else:
    display(HTML('<div class="alert alert-success">No Match Found via Pulsedive API </div>'))

Raw JSON - {'qid': None, 'iid': 2337312, 'indicator': '192.64.119.63', 'type': 'ip', 'risk': 'low', 'risk_recommended': 'low', 'manualrisk': 0, 'retired': 'No recent activity', 'stamp_added': '2018-08-06 03:48:26', 'stamp_updated': '2024-09-13 15:51:31', 'stamp_seen': '2024-06-10 20:28:14', 'stamp_probed': '2024-06-10 20:28:14', 'stamp_retired': '2024-09-13 15:51:31', 'recent': 0, 'submissions': 2, 'umbrella_rank': None, 'umbrella_domain': None, 'riskfactors': [{'rfid': 60, 'description': 'found in threat feeds', 'risk': 'medium'}, {'rfid': 50, 'description': 'hosted on common ISP', 'risk': 'none'}], 'redirects': {'from': [], 'to': []}, 'threats': [{'tid': 4, 'name': 'Banjori', 'category': 'malware', 'risk': 'unknown', 'stamp_linked': '2020-05-03 01:52:52'}, {'tid': 431, 'name': 'CryptoMining', 'category': 'general', 'risk': 'low', 'stamp_linked': '2020-02-21 02:54:12'}, {'tid': 67, 'name': 'Phishing', 'category': 'general', 'risk': 'high', 'stamp_linked': '2018-08-06 03:48:26'}, {'tid

5 Threats Idenfied

 PulseDive Tags or Categories Identified:
malware
general
general
general
malware

 PulseDive Reported Related Threats Description:
Banjori
CryptoMining
Phishing
qsnatch
Suppobox

 PulseDive Feed Names:
C&C IPs
ZeroDot1's Bad IPs Feed
Zerodot1's Mining IPs Feed


## ThreatMiner

In [ ]:
threatminerresults=threatminer_iplookup("35.35.35.35",rt=6)

#Code To extract essential details from raw json response
if threatminerresults['status_message'] == 'Results found.':
    print(f"Raw JSON - {threatminerresults}\n\n")
    display(HTML('<div class="alert alert-danger">Match Found via ThreatMiner API </div>'))
    print(f"{len(threatminerresults['results'])} References Idenfied")
    print(f"\n ThreatMiner File along with URL References:\n")
    for item in threatminerresults['results']:print("FileName Reference:\t"+item['filename']+ "\nURL Reference :\t"+item['URL'])
else:
     display(HTML('<div class="alert alert-success">No Match Found via ThreatMiner API </div>'))

Raw JSON - {'status_code': '200', 'status_message': 'Results found.', 'results': [{'filename': 'Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishingAttack.pdf', 'year': '2018', 'URL': 'https://www.threatminer.org/report.php?q=Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishingAttack.pdf&y=2018'}, {'filename': 'TheOilRigCampaign_AttacksonSaudiArabianOrganizationsDeliverHelminthBackdoor-PaloAltoNetworksBlogPaloAltoNetworksBlog.pdf', 'year': '2016', 'URL': 'https://www.threatminer.org/report.php?q=TheOilRigCampaign_AttacksonSaudiArabianOrganizationsDeliverHelminthBackdoor-PaloAltoNetworksBlogPaloAltoNetworksBlog.pdf&y=2016'}, {'filename': 'Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishingAttack.pdf', 'year': '2018', 'URL': 'https://www.threatminer.org/report.php?q=Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishing

6 References Idenfied

 ThreatMiner File along with URL References:

FileName Reference:	Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishingAttack.pdf
URL Reference :	https://www.threatminer.org/report.php?q=Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishingAttack.pdf&y=2018
FileName Reference:	TheOilRigCampaign_AttacksonSaudiArabianOrganizationsDeliverHelminthBackdoor-PaloAltoNetworksBlogPaloAltoNetworksBlog.pdf
URL Reference :	https://www.threatminer.org/report.php?q=TheOilRigCampaign_AttacksonSaudiArabianOrganizationsDeliverHelminthBackdoor-PaloAltoNetworksBlogPaloAltoNetworksBlog.pdf&y=2016
FileName Reference:	Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishingAttack.pdf
URL Reference :	https://www.threatminer.org/report.php?q=Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishingAttack.pdf&y=2018
FileName Refe

## Dshield

In [ ]:
dshieldresults=dshield_iplookup("128.210.157.251")

#Code To extract essential details from raw json response
if 'threatfeeds' in dshieldresults['ip'].keys():
    print(f"Raw JSON - {dshieldresults}\n\n")

    display(HTML('<div class="alert alert-danger">Match Found via DShield API </div>'))
    print(f"{len(dshieldresults['ip']['threatfeeds'])} ThreatFeeds Idenfied")
    print(f"\n Dshield ThreatFeeds Results:\n {dshieldresults['ip']['threatfeeds']}")
else:
    display(HTML('<div class="alert alert-success">No Match Found via DShiled API </div>'))

Raw JSON - {'ip': {'number': '128.210.157.251', 'count': None, 'attacks': None, 'maxdate': None, 'mindate': None, 'updated': None, 'comment': None, 'maxrisk': None, 'asabusecontact': 'abuse@purdue.edu', 'as': 17, 'asname': 'PURDUE', 'ascountry': 'US', 'assize': 418304, 'network': '128.210.0.0/16', 'threatfeeds': {'zeuscc': {'lastseen': '2019-07-08', 'firstseen': '2013-09-29'}}}}




1 ThreatFeeds Idenfied

 Dshield ThreatFeeds Results:
 {'zeuscc': {'lastseen': '2019-07-08', 'firstseen': '2013-09-29'}}


## Alienvault

In [ ]:
otxresults=otx_iplookup('109.248.222.47')

#Code To extract essential details from raw json response
if 'pulse_info' in otxresults.keys():
    print(f"Raw JSON - {otxresults}\n\n")
    display(HTML('<div class="alert alert-danger">Match Found via Alienvault API</div>'))
#     print("Match Found via Alienvault API")
    print(f"{len(otxresults['pulse_info']['pulses'])} Pulses Idenfied")
    print(f"\nAlienvault Tags identified:")
    for item in otxresults['pulse_info']['pulses']: print(f"Tags idenfied {item['tags']} from {item['name']}")
    print(f"\nReferences identified:")
    for item in otxresults['pulse_info']['pulses']: print(item['references'])
else:
    display(HTML('<div class="alert alert-success">No Match Found via Alienvault API </div>'))

Raw JSON - {'whois': 'http://whois.domaintools.com/109.248.222.47', 'reputation': 0, 'indicator': '109.248.222.47', 'type': 'IPv4', 'type_title': 'IPv4', 'base_indicator': {'id': 1458176, 'indicator': '109.248.222.47', 'type': 'IPv4', 'title': '', 'description': '', 'content': '', 'access_type': 'public', 'access_reason': ''}, 'pulse_info': {'count': 4, 'pulses': [{'id': '5c76b2acd1420a1aac451307', 'name': 'Malware Command and Control IPs', 'description': '', 'modified': '2024-02-17T07:03:55.237000', 'created': '2019-02-27T15:54:20.106000', 'tags': [], 'references': [], 'public': 1, 'adversary': '', 'targeted_countries': [], 'malware_families': [], 'attack_ids': [], 'industries': [], 'TLP': 'white', 'cloned_from': None, 'export_count': 35395, 'upvotes_count': 0, 'downvotes_count': 0, 'votes_count': 0, 'locked': False, 'pulse_source': 'web', 'validator_count': 0, 'comment_count': 8, 'follower_count': 0, 'vote': 0, 'author': {'username': 'otxrobot_ip', 'id': '79494', 'avatar_url': '/otxa

4 Pulses Idenfied

Alienvault Tags identified:
Tags idenfied [] from Malware Command and Control IPs
Tags idenfied [] from Malware Command and Control IPs  [Copied from otxrobot_ip]
Tags idenfied ['threat roundup', 'compromiseiocs', 'hashessee json', 'march', 'iocs', 'talos', 'ioc searching', 'center', 'json file', 'mitre att', 'microsoft word', 'tofsee', 'ursu', 'bladabindi'] from Threat Round up for March 10 to March 17
Tags idenfied [] from InQuest - 17-03-2023

References identified:
[]
[]
['https://blog.talosintelligence.com/threat-roundup-0310-0317/']
['https://labs.inquest.net/iocdb']
